In [236]:
from pathlib import Path
from itertools import combinations
import numpy as np
from operator import mul
from functools import reduce
from copy import deepcopy

input_file = Path(".") / "input.txt"

def print_matrix(matrix: list, title: str="Matrix"):
    if title:
        print(title.title())
    print(np.array(matrix), "\n")

class Point:
    def __init__(self, x, y, z: str):
        self._x = int(x)
        self._y = int(y)
        self._z = int(z)
    
    def __repr__(self) -> str:
        return f"Point({self._x},{self._y},{self._z})"
    
    def __eq__(self, other):
        return self._x == other._x and self._y == other._y and self._z == other._z

    def __hash__(self):
        return hash((self._x, self._y, self._z))

    def x(self) -> int:
        return self._x

    # calculate the Euclidean distance between two Points
    def distance(self, other: Point) -> np.float64:
        p1 = np.array((self._x, self._y, self._z))
        p2 = np.array((other._x, other._y, other._z))

        return np.linalg.norm(p1 - p2)

class Graph:
    def __init__(self):
        # keys are Points, values represent connections by set
        self._nodes = {}

    def __repr__(self) -> str:
        return f"Graph(size={self.size()})"

    def addAll(self, points: set) -> None:
        for point in points:
            if point in self._nodes:
                raise ValueError(f"{point} is already in the graph")
            self._nodes[point] = set()

    def connect(self, point1, point2: Point) -> None:
        self._add_node(point1, point2)
        self._add_node(point2, point1)

    def has(self, point: Point) -> bool:
        return point in self._nodes

    def _add_node(self, point1, point2: Point) -> None:
        if point1 in self._nodes:
            self._nodes[point1].add(point2)
        else:
            self._nodes[point1] = set({point2})

    def all_points(self) -> set:
        return set(self._nodes.keys())

    # Depth First Search traversal of the connected points starting from the given one
    # return the visited sub-graph as a new graph
    def traverse(self, start: Point) -> Graph:
        if not start in self._nodes:
            raise ValueError(f"Start point {start} is not part of the graph")

        graph = Graph()
        for node in self._traverse(start, []):
            graph._nodes[node] = deepcopy(self._nodes[node])
        return graph

    def _traverse(self, point: Point, visited_nodes: list) -> list:
        visited_nodes.append(point)
        for p in self._nodes[point]:
            if not p in visited_nodes:
                self._traverse(p, visited_nodes)
        return visited_nodes

    def size(self) -> int:
        return len(self._nodes)

    def merge(self, other: Graph, point1, point2: Point) -> Graph:
        graph = Graph()
        graph._nodes = deepcopy(self._nodes)
        for point, connections in other._nodes.items():
            if not point in graph._nodes:
                graph._nodes[point] = set()
            graph._nodes[point].union(connections)
        graph.connect(point1, point2)
        return graph

# take a set of points and calculate the distance between all pairs, that needs (N * N-1) / 2 calculations
# return a list of tuples (distance, PointA, PointB) sorted by increasing distance
def calculate_distance_of_point_pairs(points: set) -> list:
    result = []
    unique_pairs = list(combinations(points, 2))
    for point1, point2 in unique_pairs:
        distance = point1.distance(point2)
        result.append((distance, point1, point2))
    return sorted(result)

# connect point paris until the max_limit connections are reached
def clasterize_points(points_by_distance: list, max_limit: int) -> Graph:
    graph = Graph()
    for _, point1, point2 in points_by_distance[0:max_limit]:
        graph.connect(point1, point2)
    return graph

# connect point pairs until all are fully connected
# return the last two points connected together
def clasterize_points_until_fully_connected(points_by_distance: list, points: set) -> tuple:
    points_count = len(points)
    min_connections_required = points_count - 1
    graph = Graph()
    graph.addAll(points)
    for _, point1, point2 in points_by_distance[0:min_connections_required]:
        graph.connect(point1, point2)

    # initial claster: list of graph that are fully connected themselves
    clusters = calculate_clusters(graph)

    if len(clusters) == 1:
        return (point1, point2)

    # merge additional points to the cluster
    for _, point1, point2 in points_by_distance[min_connections_required:]:
        c1 = c2 = None
        for cluster in clusters:
            if cluster.has(point1):
                c1 = cluster
            if cluster.has(point2):
                c2 = cluster
            if c1 != None and c2 != None:
                break
        # 1. case: Point1 and Point2 are not present in any clusters, so they form a new cluster
        if c1 == None and c2 == None:
            cluster = Graph()
            cluster.connect(point1, point2)
            clusters.append(cluster)
        # 2. case: Point1 is present in a custer, but Point2 is not, so Point2 is added to the existing cluster
        elif c1 != None and c2 == None:
            c1.connect(point1, point2)
        # 3. case: Point1 is not in a custer, but Point2 is, so Point1 is added to the existing cluster
        elif c1 == None and c2 != None:
            c2.connect(point1, point2)
        # 4. case: both points are in an separate cluster, so these should be merged into a new cluster
        elif c1 != c2:
            cluster = c1.merge(c2, point1, point2)
            clusters.remove(c1)
            clusters.remove(c2)
            clusters.append(cluster)
        if len(clusters) == 1:
            break

    return (point1, point2)

def calculate_clusters(graph: Graph) -> list:
    points = graph.all_points()
    clusters = []
    while len(points) > 0:
        point = points.pop()
        sub_graph = graph.traverse(point)
        clusters.append(sub_graph)
        for point in sub_graph.all_points():
            points.discard(point)
    return clusters

def calculate_clusters_size(clusters: list) -> list:
    return list(map(lambda graph: graph.size(), clusters))

points = set() # store input as set of Points
with input_file.open(mode="r", encoding="utf-8") as file:
    for line in file:
        points.add(Point(*line.strip().split(",")))

points = frozenset(points)
graph = clasterize_points(calculate_distance_of_point_pairs(points), len(points))
cluster_sizes = calculate_clusters_size(calculate_clusters(graph))
#print(cluster_sizes)

point1, point2 = clasterize_points_until_fully_connected(calculate_distance_of_point_pairs(points), points)

print(f"Part1 answer: {reduce(mul, cluster_sizes[0:3])}")
print(f"Part2 answer: {point1.x() * point2.x()}")



Part1 answer: 3000
Part2 answer: 25325968


In [105]:
sorted([ (5, Point(10,5,5)), (2, Point(6,4,2)), (1, Point(6,6,6))])

[(1, Point(6,6,6)), (2, Point(6,4,2)), (5, Point(10,5,5))]

In [27]:
Point(1,2,3) == Point(1,2,3)

True

In [53]:
s1 = set({Point(1,2,3), Point(4,5,6)})
s2 = set({Point(7,8,9), Point(0,0,0), Point(6,6,6)})
l = list([s1, s2])
print(list(map(len, l)))

[2, 3]


In [205]:
[20, 6, 3, 5, 3, 2][3:]

[5, 3, 2]

In [215]:
for k,v in { Point(1,1,1): set({Point(1,2,3)}) }.items():
    print(k)
    print(v)

Point(1,1,1)
{Point(1,2,3)}
